In [2]:
from pathlib import Path
import polars as pl
from datasets import Dataset, concatenate_datasets, load_from_disk
import numpy as np
from autofaiss import build_index
from src.item_matching.build_index.model import Model

In [5]:
path = Path.home() / 'Downloads/cb'
path_db = path / 'cb_2024-03-07.parquet'

col = ['item_id', 'item_name']
df_db = (
    pl.read_parquet(path_db)
    # .sort('item_id')
    # .select(pl.col(c).name.prefix('db_') for c in col)
    .head(10_000)
)

In [18]:
items = df_db['item_name'].unique().to_list()
items[:6]

In [7]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)

In [22]:
lst_pair = [(i , v) for i in items[:2] for v in items]
lst_pair[:6]

In [23]:
score = reranker.compute_score(lst_pair, normalize=True, max_length=80)

In [25]:
data = (
    pl.DataFrame(lst_pair).transpose()
    .with_columns(pl.Series(values=score, name='score'))
    .sort(['score'], descending=True)
)
data